# FDCHP Data Processing

This Notebook demonstrates how to process FDCHP data to calculate fluxes using the tools in the `ooi-data-explorations.python.examples.fdchp` library. The FDCHP instrument is on for 20 minutes out of each hour. While the instrument is recording, it includes 10 Hz 3-axis sonic wind data, as well as buoy motion data. Here, we calculate hourly motion-corrected momentum flux from these 20-minute data segments. 

To do the calculations, we need to set up a python environment that contains the necessary dependencies.

Alongside the base repository containing this Notebook (the location of the `ooi-data-explorations` repository, clone the `mi-instrument` repo 

```
git clone https://github.com/joffreyp/mi-instrument.git
```

Create a new python virtual environment:

```
cd ~
mkdir venv
python -m venv ~/venv
source venv/bin/activate
```

Install mi-instrument requirements:

```
pip install -r mi-instrument/requirements_py3.txt
```

Install `ooi-data-explorations` and `mi-instruments`: 

```
pip install -e ooi-data-explorations/python/
pip install -e mi-instrument/
```

Prepare environment for use by Jupyter Notebook kernel:

```
pip install ipykernel
pip install ipympl
python -m ipykernel install --user --name=fdchp
vi ~/.local/share/jupyter/kernels/fdchp/kernel.json
```

Check that the first line of the `argv` array in `kernel.json` points to the python from the new virtual environment: `"/home/jovyan/venv/bin/python",`

Now, select the `Kernel` menu, click `Change kernel...` and select the new `fdchp` environment.


## Import necessary libraries

In [1]:
import os
from glob import glob
import re

import pandas as pd
from datetime import datetime, timedelta, timezone
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from examples.fdchp import fdchp_utils
from examples.fdchp.fdchp_utils import particles_to_pandas, read_file, read_file_to_pandas, convert_data_to_nwu
from examples.fdchp.process_fdchp import process_fdchp, process_fdchp_xarray
from examples.fdchp.plot_fdchp import plot_x_velocity_vs_wind_speed, plot_wave_height_vs_wind_speed

from ooi_data_explorations.common import kdata_collect_from_file_list
from ooi_data_explorations.common import load_kdata, m2m_request, m2m_collect


## Get reference designator, stream, method, start and end dates if specified
Some links provide reference designator, stream, method, start and end date information. These end up stored as environment variables. Here, we extract them and set them as python variables in this notebook. 

If these parameters were not set, they default to the recovered data for the `GI01SUMO-SBD12-08-FDCHPA000` instrument for the month of September 2023.

In [2]:
refdes = os.environ.get('ooiparams_refdes', 'GI01SUMO-SBD12-08-FDCHPA000')
stream = os.environ.get('ooiparams_stream', 'fdchp_a_instrument_recovered')
method = os.environ.get('ooiparams_method', 'recovered_inst');
start_time = datetime.fromisoformat(os.environ.get('ooiparams_startTime', '2023-09-01T00:00:00.000Z')).astimezone(timezone.utc)
end_time = datetime.fromisoformat(os.environ.get('ooiparams_endTime', '2023-09-30T23:59:59.999Z')).astimezone(timezone.utc)
print("refdes: {}, stream: {}, method: {}, start_time: {}, end_time: {}".format(refdes, stream, method, start_time, end_time))

refdes: GI01SUMO-SBD12-08-FDCHPA000, stream: fdchp_a_instrument_recovered, method: recovered_inst, start_time: 2023-09-01 00:00:00+00:00, end_time: 2023-09-30 23:59:59.999000+00:00


### Defining functions for filtering NetCDF files based on data dates contained in the files

In [3]:
def get_file_dates(filename, stream):
    # deployment0010_GI01SUMO-SBD12-08-FDCHPA000-recovered_inst-fdchp_a_instrument_recovered_20231222T210200.048000-20240107T202200.041000.nc
    dates = filename.split('_')[-1].split('.nc')[0].split('-')
    return datetime.fromisoformat(dates[0]).astimezone(timezone.utc), datetime.fromisoformat(dates[1]).astimezone(timezone.utc)

def limit_files(file_list, start_date, end_date):
    files = []
    for file in file_list:
        start, end = get_file_dates(file, stream)
        if start <= end_date and end >= start_date:
            files.append(file)
    return files

### Selecting all files for this instrument

In [4]:
refdes_parts = refdes.split('-')
site = refdes_parts[0]
node = refdes_parts[1]
sensor = '-'.join(refdes_parts[2:])

dataset_id = '-'.join([site, node, sensor, method, stream])
kdata = os.path.abspath(os.path.join(os.path.expanduser('~'), 'ooi/kdata-backups/exclude_folder_removal'))
kdata = os.path.abspath(os.path.join(kdata, dataset_id))

# Get all netcdf files for site, node, sensor, method, stream
files = files = glob(kdata + '/' + 'deployment*' + dataset_id + '*.nc')
# print(len(files))
# filter files based on start and end times
file_list = limit_files(files, start_time, end_time)
# for file in file_list:
#     print(file)

### Open NetCDF files as xarray datasets, setting `time` as the primary dimension

In [5]:
xarrays = [ xr.open_dataset(file).swap_dims({"obs": "time"}) for file in file_list ]

NameError: name 'file_list' is not defined

### Determine which variables can be dropped before merging datasets, and trim variables from datasets

In [ ]:
vars_to_drop = [var for var in xarrays[0].variables if var.startswith('fdchp_a')]
vars_to_drop += ['obs', 'year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond']
trimmed_datasets = []
for ds in xarrays:
    trimmed_datasets.append(ds.drop_vars(vars_to_drop, errors="ignore"))

### Combine trimmed datasets using the `time` dimension and subselect data between start and end times.
Also, rotate dataset to NWU coordinates

In [ ]:
data = xr.concat(trimmed_datasets, dim='time')
start_time_64 = pd.Timestamp(start_time).to_datetime64()
end_time_64 = pd.Timestamp(end_time).to_datetime64()
data = data.sel(time=slice(start_time_64, end_time_64))
convert_data_to_nwu(data)

In [ ]:
# The below may work fine in the future, but given the complexity of the multiple time dimensions in the current recovered netcdfs xarray's broadcasting is overwhelmed and it thinks it needs hundreds of TiB of memory to do the dataset merge.
# data = kdata_collect_from_file_list(file_list)

### Define xarray dataset processing to get motion-corrected fluxes from fdchp data
Note that the function `process_fdchp_xarray` is in the `ooi-data-explorations/python/examples/fdchp/process_fdchp.py` module. It calls many functions in the `ooi-data-explorations/python/examples/fdchp/fdchp_utils.py` module. See those python modules for details on how the computation is performed, and to potentially modify the filtering as desired.

In [ ]:
def process_xarray(dataset, output_filepath='fluxes'):
    # Set up some variables for FDCHP processing depending on sensor location
    subsite = dataset.subsite
    if subsite.startswith('PA'):
        lat=40.1334        #Pioneer NES
        instrument_rel_position=[-0.75, 0, -5]
    elif subsite.startswith('IS'):
        lat=59.9337        #Irminger Sea
        instrument_rel_position=[-0.75, 0, -6] 
    else:
        lat=44.6393        #Endurance
        instrument_rel_position=[-0.5, -0.5, -5]
    
    output_filename = "fluxes{}"
    errors = {}
    U=[]
    uw=[]
    sigH=[]
    times=[]
    data = []
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)

    print("Start time: {}".format(datetime.now()))
    data_processed_accumulator = timedelta(0)
    
    start = datetime.now()
    incr = 0
    #*****************************************
    # Compute flux data
    #***************************************** 
    hourly_groups = dataset.resample(time="1h", offset=-timedelta(minutes=10)) #offsetting by 10 minutes because a few datasets sometimes start before the hour, but typically end around 22 minutes into the hour.
    tot_hours = len(hourly_groups)
    for group_name, group_ds in hourly_groups:
        try:
            fluxes, Uearth, waveheight = process_fdchp_xarray(group_ds, lat, instrument_rel_position)
        except Exception as e:
            # Error processing data; probably too few datapoints
            print("Error processing fdchp dataset: {}".format(e))
            errors[filename] = e
            continue
        finally:
            data_processed_accumulator = data_processed_accumulator + (datetime.now()-start)
            incr = incr + 1
            if incr %24 == 0:
                print("File processing finished for hour {} of {}. Process time: {}".format(incr, tot_hours, data_processed_accumulator))
        
        uw = uw + [fluxes[0]]        # Fluxes: uw vw wT
        U = U + [Uearth]              # Wind speed relative to earth  
        sigH = sigH + [waveheight]    # Significant wave height
        times = times + [group_ds['time'].mean().to_numpy()]
    
    uw = np.array(uw)
    U = np.array(U)
    sigH = np.array(sigH)
    times = np.array(times)
    
    print("{} errors during processing.".format(len(errors)))
    print("End time: {}".format(datetime.now()))
    print("Processed {} hours of data in {}.".format(tot_hours, datetime.now() - start))

    return uw, U, sigH, times, errors

## Process data to calculate fluxes

In [ ]:
uw, U, sigH, times, errors =  process_xarray(data)

In [ ]:
plot_x_velocity_vs_wind_speed(U, uw)
plot_wave_height_vs_wind_speed(U, sigH, x_max = 20.0, y_max=8.0)

## Get Telemetered data from kdata directory

Find all deployments, assemble and join datasets. This presumes that the initial datasets were recovered of some kind. If not, modify the code below to pick a different method (replace the 'telemetered' parameter in `load_kdata` below).

The telemetered datasets include the fluxes calculated onboard the FDCHP instrument with flux values telemetered each hour.

In [ ]:
from ooi_data_explorations.common import load_kdata

deployments = {}
for file in file_list:
    deployment = file.split('deployment')[-1][:4]
    deployments['deployment'+deployment] = file

print("Deployments: {}".format(deployments.keys()))

# load datasets
telemetered_datasets = []
for deployment in deployments.keys():
    telemetered_datasets.append(load_kdata(site, node, sensor, 'telemetered', 'fdchp_a_dcl_instrument', tag= deployment + '*.nc'))

# join datasets
telemetered_dataset = xr.concat(telemetered_datasets, dim='time')

telemetered_dataset = telemetered_dataset.sel(time=slice(start_time_64, end_time_64))

## Plot calculated vs telemetered

In [ ]:
from matplotlib.collections import PathCollection
from matplotlib.legend_handler import HandlerPathCollection, HandlerLine2D
%matplotlib inline
# %matplotlib ipympl

marker_size = 3
legend_marker_size = 20
tick_size = 14

def update(handle, orig):
    handle.update_from(orig)
    handle.set_sizes([legend_marker_size])
    
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(times, uw, marker='o',linestyle='solid',markersize=marker_size, color='green', alpha=0.5, label="Computed")

scatter = ax.plot(telemetered_dataset['time'], telemetered_dataset['uw_momentum_flux'], marker='o',linestyle='solid', color='orange', markersize=marker_size, alpha=0.5, label="Telemetered")
ax.set_xlabel('Time', fontsize=22)
ax.set_xlim(np.min(times), np.max(times))
ax.set_ylabel('uw Along-wind momentum flux', fontsize=22)
ax.set_ylim(-1.0, 0.05)
legend = ax.legend(loc="lower right", fontsize=tick_size)

plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.title("Along-Wind Momentum Flux Comparison", fontsize=24)
# plt.savefig('calculated_vs_telemetered_comparison.png', bbox_inches='tight')
plt.show()